In [1]:
import pandas as pd
import subprocess
import os
import shutil

# 复制原CSV文件为新的CSV文件
original_csv = 'text.csv'
modified_csv = 'textmodify.csv'
shutil.copyfile(original_csv, modified_csv)

# 读取修改后的CSV文件
df = pd.read_csv(modified_csv)

# 添加新的一栏 ifcombine，默认值为 0
df['ifcombine'] = 0

# 保存修改到同一个文件
df.to_csv(modified_csv, index=False)

In [12]:
import pandas as pd
import os
import subprocess

# 读取修改后的CSV文件
modified_csv = 'textmodify.csv'
df = pd.read_csv(modified_csv)

# 创建结果文件夹和子文件夹
result_dir = "result"
result_pre_dir = os.path.join(result_dir, "result_pre")
os.makedirs(result_dir, exist_ok=True)
os.makedirs(result_pre_dir, exist_ok=True)

concat_file_list = []
previous_combine = False
start_index_for_merge = None

for index, row in df.iterrows():
    video_name = row['name']
    start_time = max(0, float(row['start']) - 0.5)
    end_time = float(row['end']) + 0.5

    if row['check'] == 1:
        output_name = os.path.join(result_pre_dir if row['ifcombine'] == 1 else result_dir, f"{index + 1}.MP4")
        
        if row['ifcombine'] == 1:
            # 保存文件名而非全路径
            concat_file_list.append(f"{index + 1}.MP4")
            if not previous_combine:
                start_index_for_merge = index + 1
            previous_combine = True
        else:
            if previous_combine:
                print("Merging videos...")
                with open(os.path.join(result_pre_dir, 'filelist.txt'), 'w') as f:
                    for file_name in concat_file_list:
                        f.write(f"file '{file_name}'\n")
                merge_result = subprocess.run([
                    'ffmpeg',
                    '-f', 'concat',
                    '-safe', '0',
                    '-i', os.path.join(result_pre_dir, 'filelist.txt'),
                    '-c', 'copy',
                    '-y',
                    os.path.join(result_dir, f"merged_{start_index_for_merge}.MP4")
                ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                print("FFmpeg Output:", merge_result.stdout.decode())
                print("FFmpeg Errors:", merge_result.stderr.decode())
                concat_file_list = []
                previous_combine = False
                start_index_for_merge = None

        if os.path.exists(video_name):
            subprocess.run([
                'ffmpeg',
                '-ss', str(start_time),
                '-to', str(end_time),
                '-i', video_name,
                '-c:v', 'libx264',
                '-preset', 'fast',
                '-crf', '22',
                '-c:a', 'aac',
                '-vsync', '1',
                '-async', '1',
                '-y',
                output_name
            ])
        else:
            print(f"File {video_name} does not exist")

if previous_combine:
    print("Merging final segment of videos...")
    with open(os.path.join(result_pre_dir, 'filelist.txt'), 'w') as f:
        for file_name in concat_file_list:
            f.write(f"file '{file_name}'\n")
    final_merge_result = subprocess.run([
        'ffmpeg',
        '-f', 'concat',
        '-safe', '0',
        '-i', os.path.join(result_pre_dir, 'filelist.txt'),
        '-c', 'copy',
        '-y',
        os.path.join(result_dir, f"final_merged_{start_index_for_merge}.MP4")
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("FFmpeg Output:", final_merge_result.stdout.decode())
    print("FFmpeg Errors:", final



